<a href="https://colab.research.google.com/github/mohmaed7777/Feedback-Prize---Predicting-Effective-Arguments/blob/main/Feedback_prize_logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [2]:
!mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download -c feedback-prize-effectiveness

  0% 0.00/8.13M [00:00<?, ?B/s]
100% 8.13M/8.13M [00:00<00:00, 349MB/s]


In [ ]:
! unzip feedback-prize-effectiveness.zip

In [8]:
# importing step: 
import numpy as np 
import pandas  as pd 
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [10]:
NFOLDs = 10

In [11]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [13]:
train_df['text'] = train_df['essay_id'].apply(lambda x: open(f'/content/train/{x}.txt').read())
test_df['text'] = test_df['essay_id'].apply(lambda x: open(f'/content/test/{x}.txt').read())


In [15]:
train_df.tail()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,text
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,Some people may ask multiple people for advice...
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,Some people may ask multiple people for advice...
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,Some people may ask multiple people for advice...
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,Some people may ask multiple people for advice...
36764,c8a973681feb,FFA381E58FC6,In conclusion asking for an opinion can be ben...,Concluding Statement,Ineffective,Some people may ask multiple people for advice...


In [16]:
effectiveness_map = {'Ineffective':0, 'Adequate':1, 'Effective':2}
train_df['target'] = train_df['discourse_effectiveness'].map(effectiveness_map)

In [17]:
train_df = train_df.reset_index(drop=True)

In [20]:
skfold = StratifiedKFold(n_splits=NFOLDs,shuffle=True,random_state=NFOLDs)
for i,(train_index, test_index) in enumerate(skfold.split(train_df, train_df["target"])):
    train_df.loc[test_index,"fold"] = i
print(train_df.fold.value_counts())   

3.0    3677
2.0    3677
1.0    3677
4.0    3677
0.0    3677
8.0    3676
9.0    3676
6.0    3676
5.0    3676
7.0    3676
Name: fold, dtype: int64


In [ ]:
preds = []

In [ ]:
for n_fold in range(NFOLDs):
  train_df = train_df[train_df['fold'] != n_fold]
  dataset_eval_= train_df[train_df['fold'] == n_fold]

  tf = TfidfVectorizer(ngram_range=(1,2), norm='l2', smooth_idf=True)
  tr_dicourse_tfidf = tf.fit_transform(train_df['discourse_text'])
  eval_discourse_tfidf = tf.fit_transform(dataset_eval_['discourse_text'])
  te_discourse_tfidf = tf.transform(test_df['discourse_text'])

  # Text : 
  tf = TfidfVectorizer(ngram_range=(1,2), norm='l2', smooth_idf=True)
  tf_text_tfidf = tf.fit_transform(train_df['text'])
  eval_text_tfidf = tf.fit_transform(dataset_eval_['text'])
  te_text_tfidf = tf.fit_transform(test_df['text'])

  # discourse_type : 
  ohe = OneHotEncoder()
  train_type_ohe = sparse.crs_matrix(ohe.fit_transform(train_df['discourse_type'].values.reshape(-1,1)))
  eval_type_ohe = sparse.csr_matrix(ohe.transform(dataset_eval_['discourse_type'].values.reshape(-1,1)))
  test_type_ohe =  sparse.csr_matrix(ohe.transform(test_df["discourse_type"].values.reshape(-1,1)))

  # Stack each vector representions : 
  tr_tfidf = sparse.hstack((train_type_ohe, ))


